Breast Cancer Prediction Using Neural Networks in PyTorch

Requirements

In [179]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Device Configuration

In [180]:
# Check for CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


Data Collection and Preprocessing

In [181]:
# Load the breast cancer dataset
data = load_breast_cancer()
X = data.data
y = data.target

In [182]:
# Split the dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [183]:
# Standardize the data using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [184]:
# Convert data to PyTorch tensors and move it to GPU
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

Neural Network Architecture

In [185]:
# Define the neural network architecture
class NeuralNet(nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

In [186]:
# Define hyperparameters
input_size = X_train.shape[1]
hidden_size = 64  # neurons
output_size = 1  # single neuron
learning_rate = 0.001
num_epochs = 100

In [187]:
# Initialize the neural network and move it to the GPU
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [188]:
# Define the loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Training the Neural Network

In [189]:
# Training the model
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train.view(-1, 1))
    loss.backward()
    optimizer.step()

    # calculate accuracy
    with torch.no_grad():
        predicted = outputs.round()
        correct = (predicted == y_train.view(-1, 1)).float().sum()
        accuracy = correct / y_train.size(0)

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1} / {num_epochs}], Loss : {loss.item():.4f}, Accuracy: {accuracy.item() * 100:.2f}%")

Epoch [10 / 100], Loss : 0.5797, Accuracy: 73.85%
Epoch [20 / 100], Loss : 0.4508, Accuracy: 90.99%
Epoch [30 / 100], Loss : 0.3543, Accuracy: 92.75%
Epoch [40 / 100], Loss : 0.2825, Accuracy: 93.63%
Epoch [50 / 100], Loss : 0.2302, Accuracy: 94.51%
Epoch [60 / 100], Loss : 0.1928, Accuracy: 95.82%
Epoch [70 / 100], Loss : 0.1659, Accuracy: 96.26%
Epoch [80 / 100], Loss : 0.1461, Accuracy: 96.48%
Epoch [90 / 100], Loss : 0.1310, Accuracy: 97.36%
Epoch [100 / 100], Loss : 0.1192, Accuracy: 97.36%


Model Evaluation

In [190]:
# Evaluation on training set
model.eval()
with torch.no_grad():
    outputs = model(X_train)
    predicted = outputs.round()
    correct = (predicted == y_train.view(-1, 1)).float().sum()
    accuracy = correct / y_train.size(0)
    print(f"Accuracy on training data: {accuracy.item() * 100:.2f}%")

Accuracy on training data: 97.36%


In [191]:
# Evaluation on testing set
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    predicted = outputs.round()
    correct = (predicted == y_test.view(-1, 1)).float().sum()
    accuracy = correct / y_test.size(0)
    print(f"Accuracy on testing data: {accuracy.item() * 100:.2f}%")

Accuracy on testing data: 97.37%
